In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import LongformerTokenizer, LongformerForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
import torch
import numpy as np
from sklearn.metrics import accuracy_score

c:\Users\rache\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset
df = pd.read_csv("combined_output.csv", on_bad_lines='skip')

In [3]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,'5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d...,'kabargayo.com,'Kredit sepeda motor bisa terbayar jika Anda m...,'https://www.kabargayo.com/2024/09/19/kredit-s...,"'Jakarta, VIVA – Pembayaran kredit sepeda moto...",19/09/2024 22.32,'id,19/09/2024 22.32,'Aldi Hadad,'https://i1.wp.com/thumb.viva.co.id/media/fron...,NaN,'positive,NaN,"'Hari Pembayaran Berbayar atau Harcilnas 2024,...",NaN,5250000,Adira,NaN,NaN
1,'e1e3f8d68b58568e8217b7562d48de634fceb0d837135...,'viva.co.id,'Kredit Motor Bisa Lunas Jika Bayar Cicilan Te...,'https://www.viva.co.id/otomotif/tips/1753596-...,"'Jakarta, VIVA – Cicilan kredit motor yang ser...",19/09/2024 22.30,'id,19/09/2024 22.30,'Krisna Wicaksono,'https://thumb.viva.co.id/media/frontend/thumb...,NaN,'positive,NaN,"'Harinya Cicilan Lunas,2024,PT Adira Dinamika ...",NaN,5250000,Adira,NaN,NaN
2,'dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf...,'kabarmegapolitan.pikiran-rakyat.com,'Adira Finance Umumkan Pemenang HARCILNAS 2024...,'https://kabarmegapolitan.pikiran-rakyat.com/b...,'KABARMEGAPOLITAN.com - PT Adira Dinamika Mult...,19/09/2024 21.45,'id,19/09/2024 21.45,'Yuliansyah,'https://assets.pikiran-rakyat.com/www/network...,NaN,'positive,'HARCILNAS merupakan wujud apresiasi kami kepa...,"'PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 ...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,'56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b...,'banggairaya.id,"'Dapatkan Promo Menarik, Yamaha Prima Motor Ra...",'https://banggairaya.id/dapatkan-promo-menarik...,'BANGGAI RAYA- Yamaha Prima Motor ramaikan pam...,19/09/2024 19.45,'id,19/09/2024 19.45,'Chikal Connect,'https://i0.wp.com/banggairaya.id/wp-content/u...,NaN,'neutral,NaN,"'RAYA- Yamaha Prima Motor,Banggai Goverment Ex...",NaN,5250000,Adira,NaN,NaN
4,'1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721...,'jakarta.tribunnews.com,"'Sindikat Penipuan Leasing, Satu Bulan Ajukan ...",'https://jakarta.tribunnews.com/2024/09/19/sin...,'Laporan wartawan TribunJakarta.com Yusuf Bach...,19/09/2024 18.43,'id,19/09/2024 18.43,'Yusuf Bachtiar,'https://asset-2.tstatic.net/jakarta/foto/bank...,NaN,'neutral,'Pelaku ini melakukan pembiayaan pembelian ken...,"'Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [4]:
# Clean extra characters from all columns in the DataFrame
for column in df.columns:
    if df[column].dtype == 'object':  # Check if the column is of string type
        df[column] = df[column].str.strip("'")  # Remove extra characters

# Filter relevant columns
df_filtered = df[['body', 'sentiment']].dropna()


In [5]:
df.head()

,original_id,source_name,title,url,body,date_published,language,date_modified,author_list,images,description,sentiment,emotions,entities,quotations,prValues,clipping,label,category
0,5404c9d2fd24852afa122f2cc01cb3acba3c5d05b682d4...,kabargayo.com,Kredit sepeda motor bisa terbayar jika Anda me...,https://www.kabargayo.com/2024/09/19/kredit-se...,"Jakarta, VIVA – Pembayaran kredit sepeda motor...",19/09/2024 22.32,id,19/09/2024 22.32,Aldi Hadad,https://i1.wp.com/thumb.viva.co.id/media/front...,NaN,positive,NaN,"Hari Pembayaran Berbayar atau Harcilnas 2024,P...",NaN,5250000,Adira,NaN,NaN
1,e1e3f8d68b58568e8217b7562d48de634fceb0d8371356...,viva.co.id,Kredit Motor Bisa Lunas Jika Bayar Cicilan Tep...,https://www.viva.co.id/otomotif/tips/1753596-k...,"Jakarta, VIVA – Cicilan kredit motor yang seri...",19/09/2024 22.30,id,19/09/2024 22.30,Krisna Wicaksono,https://thumb.viva.co.id/media/frontend/thumbs...,NaN,positive,NaN,"Harinya Cicilan Lunas,2024,PT Adira Dinamika M...",NaN,5250000,Adira,NaN,NaN
2,dca74b8fa4eabf60cebfa7b811ecb385872a0fd301eaf5...,kabarmegapolitan.pikiran-rakyat.com,Adira Finance Umumkan Pemenang HARCILNAS 2024:...,https://kabarmegapolitan.pikiran-rakyat.com/bi...,KABARMEGAPOLITAN.com - PT Adira Dinamika Multi...,19/09/2024 21.45,id,19/09/2024 21.45,Yuliansyah,https://assets.pikiran-rakyat.com/www/network/...,NaN,positive,HARCILNAS merupakan wujud apresiasi kami kepad...,"PT Adira Dinamika,Cicilan Lunas HARCILNAS,12 p...","(Person :Tania Endah Budhi ,Quote : HARCILNAS ...",5250000,Adira,NaN,NaN
3,56c73e2a6d254a17a5cc21dee7ed0b4660c3af70c093b2...,banggairaya.id,"Dapatkan Promo Menarik, Yamaha Prima Motor Ram...",https://banggairaya.id/dapatkan-promo-menarik-...,BANGGAI RAYA- Yamaha Prima Motor ramaikan pame...,19/09/2024 19.45,id,19/09/2024 19.45,Chikal Connect,https://i0.wp.com/banggairaya.id/wp-content/up...,NaN,neutral,NaN,"RAYA- Yamaha Prima Motor,Banggai Goverment Exp...",NaN,5250000,Adira,NaN,NaN
4,1cd6c6db60224b6ee5f49cd5d6c62cd2850d9f3255721d...,jakarta.tribunnews.com,"Sindikat Penipuan Leasing, Satu Bulan Ajukan K...",https://jakarta.tribunnews.com/2024/09/19/sind...,Laporan wartawan TribunJakarta.com Yusuf Bacht...,19/09/2024 18.43,id,19/09/2024 18.43,Yusuf Bachtiar,https://asset-2.tstatic.net/jakarta/foto/bank/...,NaN,neutral,Pelaku ini melakukan pembiayaan pembelian kend...,"Yusuf Bachtiar TRIBUNJAKARTACOM,MEDAN,SATRIA,T...","(Person :Dedi ,Quote : Pelaku ini melakukan pe...",5250000,Adira,NaN,NaN


In [6]:
# Split the data into training and validation sets (80% training, 20% validation)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_filtered['body'].tolist(), df_filtered['sentiment'].tolist(), test_size=0.2, random_state=42
)

In [7]:
# Load the Longformer tokenizer
tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

# Tokenize the training and validation texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


In [8]:
# Map sentiment labels to integers
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
train_labels = [label_mapping[label] for label in train_labels]
val_labels = [label_mapping[label] for label in val_labels]

In [9]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [10]:
# Create datasets
train_dataset = SentimentDataset(train_encodings, train_labels)
val_dataset = SentimentDataset(val_encodings, val_labels)

In [11]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [16]:
# Load Longformer model for sequence classification
model = LongformerForSequenceClassification.from_pretrained("allenai/longformer-base-4096", num_labels=3)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
# Set training arguments with improvements
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy="epoch",           # Evaluate at the end of each epoch
    save_strategy="epoch",                 # Save at the end of each epoch
    per_device_train_batch_size=8,         # Increase batch size
    per_device_eval_batch_size=8,          # Increase batch size
    num_train_epochs=5,                    # Increase the number of training epochs
    learning_rate=2e-5,                    # Decrease the learning rate
    weight_decay=0.01,                     # Add weight decay for regularization
    logging_dir='./logs',
    logging_steps=50,                      # Log less frequently
    save_total_limit=2,                    # Limit the total number of saved checkpoints
    load_best_model_at_end=True,           # Load the best model at the end of training
    metric_for_best_model='eval_loss',     # Save based on the best eval loss
    greater_is_better=False,               # Lower eval loss is better
    lr_scheduler_type="cosine",            # Use cosine learning rate schedule
)

In [18]:
# Define Trainer with early stopping
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],  # Stop early if no improvement
)

In [19]:
# Train the model
trainer.train()

  1%|          | 50/4500 [07:06<10:28:57,  8.48s/it]

{'loss': 1.0608, 'grad_norm': 4.35317325592041, 'learning_rate': 1.999390827019096e-05, 'epoch': 0.06}


  2%|▏         | 100/4500 [14:17<10:30:58,  8.60s/it]

{'loss': 0.9961, 'grad_norm': 5.180014133453369, 'learning_rate': 1.9975640502598243e-05, 'epoch': 0.11}


  3%|▎         | 150/4500 [21:26<10:09:11,  8.40s/it]

...

                                                      
100%|██████████| 4500/4500 [11:19:02<00:00,  8.91s/it]

{'eval_loss': 0.7674257755279541, 'eval_runtime': 79.7063, 'eval_samples_per_second': 22.595, 'eval_steps_per_second': 2.835, 'epoch': 5.0}


100%|██████████| 4500/4500 [11:19:06<00:00,  9.05s/it]

{'train_runtime': 40746.4594, 'train_samples_per_second': 0.884, 'train_steps_per_second': 0.11, 'train_loss': 0.5006378892262777, 'epoch': 5.0}


TrainOutput(global_step=4500, training_loss=0.5006378892262777, metrics={'train_runtime': 40746.4594, 'train_samples_per_second': 0.884, 'train_steps_per_second': 0.11, 'total_flos': 1.1823414054912e+16, 'train_loss': 0.5006378892262777, 'epoch': 5.0})

In [23]:
# Save the trained model and tokenizer
model.save_pretrained('./longformer_model')
tokenizer.save_pretrained('./longformer_model')

('./longformer_model\\tokenizer_config.json',
 './longformer_model\\special_tokens_map.json',
 './longformer_model\\vocab.json',
 './longformer_model\\merges.txt',
 './longformer_model\\added_tokens.json')

In [20]:
# Evaluate the model
results = trainer.evaluate()

100%|██████████| 226/226 [01:19<00:00,  2.86it/s]


In [21]:
print(results)

{'eval_loss': 0.6224883794784546, 'eval_runtime': 79.6042, 'eval_samples_per_second': 22.624, 'eval_steps_per_second': 2.839, 'epoch': 5.0}


In [22]:
# After training, get predictions on the validation dataset
predictions = trainer.predict(val_dataset)

# Get the predicted logits
logits = predictions.predictions

# Convert logits to predicted class labels
predicted_labels = np.argmax(logits, axis=1)

# Get the true labels
true_labels = val_labels

# Calculate accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

print(f"Accuracy: {accuracy:.4f}")

100%|██████████| 226/226 [01:19<00:00,  2.85it/s]


Accuracy: 0.7579


In [12]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cuda


In [13]:
# Load the saved model and tokenizer
model = LongformerForSequenceClassification.from_pretrained('./longformer_model').to(device)
tokenizer = LongformerTokenizer.from_pretrained('./longformer_model')

In [14]:
# Define label mapping
label_mapping = {0: "Negative", 1: "Neutral", 2: "Positive"}

In [15]:
# Testing new articles
def classify_article(article):
    # Tokenize the input article and move the tensors to the correct device
    inputs = tokenizer(article, return_tensors="pt", truncation=True, padding=True, max_length=512)
    
    # Move tokenized inputs to the correct device (GPU or CPU)
    inputs = {key: value.to(device) for key, value in inputs.items()}
    
    with torch.no_grad():
        # Perform inference using the model
        outputs = model(**inputs)
    
    logits = outputs.logits
    # Get the predicted label (index of the highest logit)
    predicted_label = np.argmax(logits.cpu().numpy(), axis=1)[0]
    
    # Calculate sentiment scores (probabilities)
    sentiment_score = torch.softmax(logits, dim=1).cpu().numpy()[0]
    
    # Map predicted label to sentiment text
    sentiment_result = label_mapping[predicted_label]
    
    return sentiment_result, sentiment_score

In [16]:
# User input for a new article
new_article = input("Your article:")
sentiment_result, sentiment_score = classify_article(new_article)
print(f"The sentiment of the article is: {sentiment_result}")
print(f"Sentiment scores: {sentiment_score}")


Initializing global attention on CLS token...


The sentiment of the article is: Positive
Sentiment scores: [0.00133929 0.01447496 0.98418576]


In [17]:
from transformers import LongformerTokenizer, EncoderDecoderModel

# Load summarization model and tokenizer
summarization_model = EncoderDecoderModel.from_pretrained("patrickvonplaten/longformer2roberta-cnn_dailymail-fp16")
summarization_tokenizer = LongformerTokenizer.from_pretrained("allenai/longformer-base-4096")

You are using a model of type encoder_decoder to instantiate a model of type encoder-decoder. This is not supported for all configurations of models and can yield errors.
EncoderDecoderModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to 

In [18]:
# Function to summarize text
def summarize_text(text):
    input_ids = summarization_tokenizer(text, return_tensors="pt", max_length=4096, truncation=True).input_ids
    output_ids = summarization_model.generate(input_ids, max_length=150, min_length=30, length_penalty=2.0, num_beams=4, early_stopping=True)
    return summarization_tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [19]:
# User input for text summarization
article_to_summarize = input("Your article:")

In [20]:
# Ensure model is moved to the device
summarization_model = summarization_model.to(device)

# Tokenize and summarize
input_ids = summarization_tokenizer(article_to_summarize, return_tensors="pt").input_ids.to(device)
output_ids = summarization_model.generate(input_ids)

# Get the summary from the output tokens
summary = summarization_tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Print summary
print("Summary of the article:")
print(summary)

Input ids are automatically padded to be a multiple of `config.attention_window`: 512


Summary of the article:
Program Harcilnas 2024 is a program ini .
programs Haribah and Adira Finance Tbk (Adira Finance)
Program Harkelnas 2024 was launched in 2009 .
Program is aimed at helping to improve the marketing strategy of the program Harcilann .


In [21]:
from transformers import BartForConditionalGeneration, BartTokenizer

# Set device (CPU or GPU)
device = "cpu"  # Use CPU instead of GPU
summarization_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)
tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")

# Article text to summarize
article_to_summarize = """
Jakarta, VIVA – Pembayaran kredit sepeda motor yang seringkali menjadi beban bagi sebagian besar masyarakat, kini dapat diatasi melalui program Hari Pembayaran Berbayar atau Harcilnas 2024 yang diselenggarakan oleh PT Adira Dinamika Multi Finance Tbk (Adira Finance). Program ini memungkinkan nasabah melunasi kredit sepeda motor atau kredit lainnya secara penuh hanya dengan menukarkan poin yang terkumpul. Pada Harcilnas periode pertama tahun ini, ada 12 nasabah yang berhasil melunasi seluruh tagihannya.
"""

# Use "summarize:" prefix
input_text = f"summarize: {article_to_summarize}"

# Tokenize the input text
input_ids = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True, max_length=512).input_ids.to(device)

# Generate the summary with BART's optimized settings
output_ids = summarization_model.generate(
    input_ids,
    max_length=150,            # Shorter summary length
    num_beams=4,               # Beam search for better quality
    early_stopping=True,
)

# Decode the generated summary output
summary = tokenizer.decode(output_ids[0], skip_special_tokens=True, clean_up_tokenization_spaces=True)
print("Summary of the article:")
print(summary)


Summary of the article:
Pembayaran kredit sepeda motor yang seringkali menjadi beban bagi sebagian besar masyarakat. Program ini memungkinkan nasabah melunasi kredit atau kredit lainnya secara penuh.


In [ ]:
# Export results to CSV
import csv

def export_results(sentiment_results, summaries):
    with open('results.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Article', 'Sentiment', 'Sentiment Scores', 'Summary'])
        for article, sentiment, scores, summary in zip([new_article, article_to_summarize], 
                                                        [sentiment_result, 'N/A'], 
                                                        [sentiment_score, 'N/A'], 
                                                        [summary, 'N/A']):
            writer.writerow([article, sentiment, scores.tolist() if isinstance(scores, np.ndarray) else scores, summary])

In [ ]:
# Call the export function
export_results(sentiment_results=[(new_article, sentiment_result, sentiment_score)], 
               summaries=[summary])
print("Results exported to 'results.csv'.")